In [289]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

import pickle
import numpy as np
import pandas as pd

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_colwidth', None)

In [290]:
with open("./datasets/mnist/train-images-2d.pkl", "rb") as f:
  train_x = pickle.load(f)

with open("./datasets/mnist/train-labels.pkl", "rb") as f:
  train_t = pickle.load(f)

with open("./datasets/mnist/test-images-2d.pkl", "rb") as f:
  test_x = pickle.load(f)

with open("./datasets/mnist/test-labels.pkl", "rb") as f:
  test_t = pickle.load(f)
  
print(train_x.shape)

(60000, 1, 28, 28)


In [291]:
class Dataset(data.Dataset):
  def __init__(self, data, label) :
    self.data = data
    self.label = label

  def __len__(self) :
    return len(self.data)

  def __getitem__(self, idx) :
    data = torch.FloatTensor(self.data[idx])
    label = torch.FloatTensor(self.label[idx])

    return data, label

In [292]:
train_dataset = Dataset(train_x, train_t)
test_dataset = Dataset(test_x, test_t)

train, validation = torch.utils.data.random_split(train_dataset, [0.9, 0.1])

train_loader = data.DataLoader(train, batch_size=100, shuffle=True)
validation_loader = data.DataLoader(validation, batch_size=100, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [293]:
# Conv2d(in_channel, out_channel, (patch_x, patch_y), stride, padding)

class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 32, 5, padding=15),
      nn.MaxPool2d(2),
      nn.ReLU(),

      nn.Conv2d(32, 32, 5),
      nn.MaxPool2d(2),
      nn.ReLU(),
      nn.Dropout(0.5),
  
      nn.Conv2d(32, 64, 5),
      nn.MaxPool2d(2),
      nn.ReLU(),
      nn.Dropout(0.5),

      nn.Flatten(1),
      nn.Linear(576, 100),
      nn.ReLU(),
      nn.Dropout(0.5),

      nn.Linear(100, 10),
      nn.Softmax(dim=1),
    )

  def forward(self, x):
    y=self.layers(x)
    return y
  
model = Model()
model

Model(
  (layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(15, 15))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
    (7): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): ReLU()
    (10): Dropout(p=0.5, inplace=False)
    (11): Flatten(start_dim=1, end_dim=-1)
    (12): Linear(in_features=576, out_features=100, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.5, inplace=False)
    (15): Linear(in_features=100, out_features=10, bias=True)
    (16): Softmax(dim=1)
  )
)

In [294]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):
  for i, (x, t) in enumerate(train_loader):
    optimizer.zero_grad()
    y = model(x)
    loss_train = criterion(y, t)
    loss_train.backward()
    optimizer.step()

    if i % 10 == 0:
      for x, t in validation_loader:
        y = model(x)
        loss_validation = criterion(y, t)

      print(f"Epoch #{epoch}, Batch #{i}, Train Loss: {loss_train.item():.4f}, Validation Loss: {loss_validation.item():.4f}")      

Epoch #0, Batch #0, Train Loss: 2.3016, Validation Loss: 2.3024
Epoch #0, Batch #10, Train Loss: 2.3003, Validation Loss: 2.2995
Epoch #0, Batch #20, Train Loss: 2.2784, Validation Loss: 2.2835
Epoch #0, Batch #30, Train Loss: 2.2122, Validation Loss: 2.2472
Epoch #0, Batch #40, Train Loss: 2.1776, Validation Loss: 2.1480
Epoch #0, Batch #50, Train Loss: 2.0313, Validation Loss: 2.0919
Epoch #0, Batch #60, Train Loss: 2.0119, Validation Loss: 1.9527
Epoch #0, Batch #70, Train Loss: 1.9933, Validation Loss: 1.9192
Epoch #0, Batch #80, Train Loss: 1.8823, Validation Loss: 1.9807
Epoch #0, Batch #90, Train Loss: 1.8466, Validation Loss: 1.9045
Epoch #0, Batch #100, Train Loss: 1.8873, Validation Loss: 1.8436
Epoch #0, Batch #110, Train Loss: 1.8527, Validation Loss: 1.7876
Epoch #0, Batch #120, Train Loss: 1.7763, Validation Loss: 1.8092
Epoch #0, Batch #130, Train Loss: 1.7613, Validation Loss: 1.8460
Epoch #0, Batch #140, Train Loss: 1.8142, Validation Loss: 1.7596
Epoch #0, Batch #150,

In [296]:
model.eval()
total = 0
correct = 0

for x, t in test_loader:
  y = model(x)
  y = torch.argmax(y, dim=1)
  t = torch.argmax(t, dim=1)

  total += t.size(0)
  correct += (y == t).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 98.13%
